In [1]:
from parser import *


class PersonTemplate(Template):
    name: str
    age: int
    sex: Optional[str]  # 如果数据中没有这一相则会被忽略
    _t: ...  # 以 _ 开头的私有方法会被忽略


class DataTemplate(Template):
    person: list[PersonTemplate]  # 数据中的 person 是个 PersonTemplate 的列表


data = JsonParser(DataTemplate, """
    {
        "person":[
            {
                "name":"toby",
                "age":"100"
            },
            {
                "name":"aaaa",
                "age":"114514",
                "sex":"哼哼哼啊啊啊啊啊"
            }
        ]
    }
    """)

data

{'person': [{'name': 'toby', 'age': 100}, {'name': 'aaaa', 'age': 114514, 'sex': '哼哼哼啊啊啊啊啊'}]}

In [2]:
data = JsonParser({
    "person":PersonTemplate
}, """
    {
        "person":[
            {
                "name":"toby",
                "age":"100"
            },
            {
                "name":"aaaa",
                "age":"114514",
                "sex":"哼哼哼啊啊啊啊啊"
            }
        ]
    }
    """)  # 使用 dict 代替 类

data

{'person': [{'name': 'toby', 'age': '100'},
  {'name': 'aaaa', 'age': '114514', 'sex': '哼哼哼啊啊啊啊啊'}]}

In [3]:
class Bndbox(Template):
    xmin: XmlElement[..., int]  # 对于 Xml 数据需要 加 XmlElement[]，因为其会先被转为 {attr:属性,value:值} 的字典，其中...表示attr的模板，而int表示value的模板
    ymin: XmlElement[..., int]
    xmax: XmlElement[..., int]
    ymax: XmlElement[..., int]


class ObjectTemplate(Template):
    name: XmlElement[..., str]
    bndbox: XmlElement[..., Bndbox]


class AnnotationTemplate(Template):
    filename:XmlElement[..., str]
    object: List[XmlElement[..., ObjectTemplate]]

xml = ElementTree.parse(
    "./resource/010001.xml")
annotation = XmlParser(XmlElement[...,AnnotationTemplate], xml)
annotation

{'attr': {}, 'value': {'filename': {'attr': {}, 'value': '010001.jpg'}, 'object': [{'attr': {}, 'value': {'name': {'attr': {}, 'value': 'person'}, 'bndbox': {'attr': {}, 'value': {'xmin': {'attr': {}, 'value': 287}, 'ymin': {'attr': {}, 'value': 428}, 'xmax': {'attr': {}, 'value': 351}, 'ymax': {'attr': {}, 'value': 662}}}}}, {'attr': {}, 'value': {'name': {'attr': {}, 'value': 'person'}, 'bndbox': {'attr': {}, 'value': {'xmin': {'attr': {}, 'value': 351}, 'ymin': {'attr': {}, 'value': 391}, 'xmax': {'attr': {}, 'value': 424}, 'ymax': {'attr': {}, 'value': 642}}}}}, {'attr': {}, 'value': {'name': {'attr': {}, 'value': 'person'}, 'bndbox': {'attr': {}, 'value': {'xmin': {'attr': {}, 'value': 466}, 'ymin': {'attr': {}, 'value': 367}, 'xmax': {'attr': {}, 'value': 550}, 'ymax': {'attr': {}, 'value': 614}}}}}, {'attr': {}, 'value': {'name': {'attr': {}, 'value': 'person'}, 'bndbox': {'attr': {}, 'value': {'xmin': {'attr': {}, 'value': 700}, 'ymin': {'attr': {}, 'value': 354}, 'xmax': {'att

In [4]:
annotation.object[0]  # XmlElement 访问属性时会去搜索.value中是否有object这个属性，若有则返回value.object，再在 .attr中搜索object.

{'attr': {}, 'value': {'name': {'attr': {}, 'value': 'person'}, 'bndbox': {'attr': {}, 'value': {'xmin': {'attr': {}, 'value': 287}, 'ymin': {'attr': {}, 'value': 428}, 'xmax': {'attr': {}, 'value': 351}, 'ymax': {'attr': {}, 'value': 662}}}}}

In [5]:
# 这个 xml 实际上并没有标签属性，更简单的方法：

class Bndbox(Template):
    xmin: int  # 对于 Xml 数据需要 加 XmlElement[]，因为其会先被转为 {attr:属性,value:值} 的字典，其中...表示attr的模板，而int表示value的模板
    ymin: int
    xmax: int
    ymax: int


class ObjectTemplate(Template):
    name: str
    bndbox: Bndbox


class AnnotationTemplate(Template):
    filename:str
    object: List[ObjectTemplate]

xml = ElementTree.parse("./resource/010001.xml")
annotation = XmlParser(AnnotationTemplate, xml,ignore_attr=True)
annotation

{'filename': '010001.jpg', 'object': [{'name': 'person', 'bndbox': {'xmin': 287, 'ymin': 428, 'xmax': 351, 'ymax': 662}}, {'name': 'person', 'bndbox': {'xmin': 351, 'ymin': 391, 'xmax': 424, 'ymax': 642}}, {'name': 'person', 'bndbox': {'xmin': 466, 'ymin': 367, 'xmax': 550, 'ymax': 614}}, {'name': 'person', 'bndbox': {'xmin': 700, 'ymin': 354, 'xmax': 761, 'ymax': 585}}, {'name': 'person', 'bndbox': {'xmin': 704, 'ymin': 517, 'xmax': 806, 'ymax': 794}}, {'name': 'person', 'bndbox': {'xmin': 1124, 'ymin': 22, 'xmax': 1196, 'ymax': 245}}]}